<a href="https://colab.research.google.com/github/ariana-caetano/python_mercado-financeiro/blob/main/20220512_formulaGreenblat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fórmula Mágica - Joel Greenblat

Joel Greenblatt escreveu um livro (“The Little Book that Beats the Market”), onde explica como os investidores podem aplicar sistematicamente uma fórmula que busca bons negócios quando eles estão disponíveis a preços de pechincha. A essa estratégia deu o nome de Fórmula Mágica.
Conheci essa fórmula no Canal do YouTube do Dica de Hoje onde o analista Daniel Nigri fez um vídeo sobre ela. E para quem quiser assistir esse é o link: https://www.youtube.com/watch?v=688h-8JhmXw

A fórmula usa dois indicadores:
* ROIC
* EV/EBIT

Baseado nesses indicadores queremos formar um ranking de papéis que possuam o menor EV/EBIT e o maior ROIC.

Essa é a forma bem simplificada, o próprio Greenblat faz algumas ressalvas e sugere mais alguns critérios para a escolha dos ativos como: 
* exclusão de bancos;
* empresas de infraestrura (utilities)
* empresas com capitalização de mercado inferior a US$ 50 milhões

Existem também uma variação da fórmula que permitiria usar o ROE e P/L, mas a lista de ativos obitidos utilizando ROIC e EV/EBIT são diferentes da lista utilizando ROE e P/L.

Mas aqui, para efeito de estudos de desenvolvimento em Python vou me ater a apenas essas duas regras: encontrar os 20 ativos que estejam com o maior ROIC e menor EV/EBIT e exibir essa lista.
O formato dessa carteira é anual. Ou seja, o investidor compra as ações indicadas hoje e só vai trocar por novas ações no próximo ano.

#### 1. Buscar os dados no site Fundamentus

In [ ]:
import requests
import pandas as pd

url = "https://www.fundamentus.com.br/resultado.php"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"}
r = requests.get(url, headers=headers)

lista = pd.read_html(r.text, decimal=",", thousands=".")
dados = lista[0]

#### 2. Tratar os dados

Nesta etapa vamos fazer um tratamento nos dados:
 
    * remover as colunas que não vamos utilizar e deixar apenas: Papel, EV/EBIT, ROIC;
    * remover as linhas que tenham EV/EBIT negativo;
    * remover as linhas com ROIC negativo;

In [ ]:
#o site fundamentus também traz os outros indicadores sugeridos pelo Greenblat 
# ROIC – Quanto maior melhor EV/EBIT – Quanto menor melhor

#deletando todas as colunas que não serão utilizadas
dados.drop(columns=['Cotação', 
                    'P/VP', 
                    'PSR',
                    'P/L', 
                    'Div.Yield',
                    'P/Ativo',
                    'P/Cap.Giro',
                    'P/EBIT',
                    'P/Ativ Circ.Liq',
                    'EV/EBITDA',
                    'Mrg Ebit',
                    'Mrg. Líq.',
                    'Liq. Corr.',
                    'ROE',
                    'Patrim. Líq',
                    'Dív.Brut/ Patrim.',
                    'Cresc. Rec.5a'], inplace=True)

#tratamento dos dados para remover as distorções
#EV/EBIT negativo 
indexNames = dados[dados['EV/EBIT'] <= 0].index
dados.drop(indexNames , inplace=True)

#garantindo que os dados analisados estão em formato numérico
#transformando as porcentagens em valores numéricos
#não fui em quem criou essa função, mas achei muito boa e resolvi adicionar ao código
def columns_which_contains(df, value):
    """
    Serch for DataFrame column's values which contains a specific value
    :value: any characters, phrases, symbols
    :returns: list of DataFrame column's names
    """
    return [df[column].name for column in df if df[column].astype(str).str.contains(value).any()]

percentual_columns = columns_which_contains(dados, '\%')

for column_name in percentual_columns:
    dados[column_name] = dados[column_name].str.replace('.', '', regex=True)
    dados[column_name] = dados[column_name].str.replace(',', '.', regex=True)
    dados[column_name] = dados[column_name].str.replace('[%,]', '', regex=True).astype(float)
    dados[column_name] = dados[column_name]/100

#remoção do ROIC negativo
#antes da tranformação dos dados do ROIC o trecho abaixo dava erro
#pois ela não era visto como numérico
indexNames = dados[dados['ROIC'] <= 0 ].index
dados.drop(indexNames , inplace=True)

#vou adicionar também um filtro de liquidez
#deixando apenas as ações com liquidez maior que 100000 nos últimos 2 meses
#seria bom checar qual o nível de liquidez aceitável para o investidor 
#conseguir entrar e sair do papelo sem problemas
indexNames = dados[dados['Liq.2meses'] < 80000].index
dados.drop(indexNames , inplace=True)


#### 3. Ranking

Agora, vamos construir um ranking das ações que se enquadram nos critérios.

A empresa com o maior ROIC recebe a nota 0, a segunda recebe nota 1, a terceira nota 2 e assim por diante. A mesma coisa é feita para o EV/EBIT, o menor recebe nota 0, o segundo menor nota 1.....

A partir dessa notas criaremos uma nova coluna com as notas SOMADAS dos dois indicadores. Aquelas com MENORES notas representam as “melhores” empresas pela combinação dos dois indicadores.

In [ ]:
#ordenação dos dados de forma descendente do ROIC
#quanto maior melhor
dados.sort_values('ROIC', ascending=False, inplace=True)

#indexando os dados seguindo a ordenação descendente
#agora o maior ROIC está no índice 0 e o menor ROIC está no último índice
dados.index = range(0,len(dados))

dados["notaROIC"] = range(0,len(dados)) 

#ordenação dos dados de forma ascendente do EV/EBIT
#quanto menor melhor
dados.sort_values('EV/EBIT', ascending=True, inplace=True)

dados["notaEV/EBIT"] = range(0,len(dados)) 


#fazendo a soma das notas

dados["somaNotas"] = 0

dfd = dados.copy()

for i in dfd.index:
    dfd.loc[i, 'somaNotas'] = dfd.loc [i, 'notaEV/EBIT']  + dfd.loc[i, 'notaROIC']

dfd.sort_values('somaNotas', ascending=True, inplace=True)

print("20 ativos - Fórmula Greenblat\n")
print(dfd.iloc[0:20])

20 ativos - Fórmula Greenblat

     Papel  EV/EBIT    ROIC    Liq.2meses  notaROIC  notaEV/EBIT  somaNotas
1    INEP4     0.70  0.9947  6.814810e+05         1            2          3
2    INEP3     0.70  0.9947  1.810030e+06         2            3          5
6    PSSA3     0.20  0.5429  3.929350e+07         6            1          7
19   SYNE3     1.16  0.3758  5.883760e+06        19            7         26
24   GOAU3     0.81  0.3587  7.232030e+05        24            4         28
23   GOAU4     0.85  0.3587  1.058960e+08        23            5         28
15   BRKM3     2.02  0.3971  4.670550e+05        15           13         28
14   BRKM5     2.10  0.3971  1.180500e+08        14           15         29
12   MRFG3     2.32  0.4181  1.285930e+08        12           20         32
5    TASA4     2.88  0.5732  2.367700e+07         5           27         32
11   VALE3     2.54  0.4249  2.655810e+09        11           22         33
9    CMIN3     2.65  0.5079  3.681940e+07         9      

#### 4. Resultado

Esse é o método sistemático e sem influência de nenuma análise um pouco mais refinada. Nem mesmo eu me aprofundei nos indicadores usados na fórmula, se está tudo correto ou não (se alguém quiser contibuir e me corrigir sobre qualquer coisa, fique a vontade).

Será que é um bom método usar apenas o ROIC e o EV/EBIT para nos indicar a compra de ações?

Mas Greenblat criou essa Fórmula Mágica e será que ela não é rentável? Que tal um backtest de como seria o desempenho dessa carteira nos útimos anos?
Esse será o próximo desafio....